In [29]:
import pandas as pd
import numpy as np

In [30]:
def initialize(nx,nh,ny):
    w1 = np.random.randn(nh,nx) *0.01
    b1 = np.zeros((nh,1))
    w2 = np.random.randn(ny,nh) * 0.01
    b2 = np.zeros((ny,1))
    
    parameters = {
        'w1':w1,
        'w2':w2,
        'b1':b1,
        'b2':b2
    }
    
    return parameters

In [33]:
def initialize_for_each_layer(layer_dim):
    L = len(layer_dim)
    parameters = {}
    for i in range(1,L):
        parameters['w'+str(i)] = np.random.randn(layer_dim[i],layer_dim[i-1])
        parameters['b'+str(i)] = np.zeros((layer_dim[i],1))
        
    return parameters

In [34]:
initialize_for_each_layer([5,4,2])

{'w1': array([[ 0.28721243,  0.7735519 , -0.16993827, -0.67995767, -0.71914896],
        [ 0.4441762 ,  0.42833011, -0.10983534,  1.02286868,  0.29372658],
        [ 1.34811674, -1.1628415 ,  0.86419431, -0.38353443,  2.62784065],
        [-0.06821729, -0.03670043, -0.47916802, -0.8733683 , -0.53979175]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.]]),
 'w2': array([[ 0.72140937,  1.69759668, -1.8415963 ,  0.43215769],
        [-0.49371679,  0.06898096, -0.27339256, -0.27935795]]),
 'b2': array([[0.],
        [0.]])}

In [35]:
def propogate(W,A,b):
    Z = np.dot(W,A) + b
    cache = (W,A,b)
    return Z, cache

In [37]:
def propogate_activation(A_prev,W,b,activation):
    if activation == 'sigmoid':
        Z,lcache = propogate(W,A_prev,b)
        A,ac_cache = sigmoid(Z)
    elif activation == 'relu':
        Z,lcache = propogate(W,A_prev,b)
        A,ac_cache = sigmoid(Z)
    cache = (lcache,ac_cache)
    return A, cache

In [ ]:
def L_model(X,parameters):
    L = len(parameters) // 2
    A = X
    cache = []
    for i in range(1,L):
        A_prev = A
        A,c = propogate_activation(A_prev,parameters['w'+str(i)],parameters['b'+str(i)],'relu')
        cache.append(c)
    AL,c = propogate_activation(A,parameters['w'+str(L)],parameters['b'+str(L)],'sigmoid')
    cache.append(c)
    return AL,cache

In [38]:
def compute_cost(AL, Y):
    m = Y.shape[1]

    # Compute loss from aL and y.
    ### START CODE HERE ### (≈ 1 lines of code)
    cost = (-1/m) * (np.dot(Y, np.log(AL).T) + np.dot((1-Y), np.log(1-AL).T))
    ### END CODE HERE ###
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

In [39]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### START CODE HERE ### (≈ 3 lines of code)
    dW = (1/m) * np.dot(dZ, A_prev.T)
    db = (1/m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T,dZ)
    ### END CODE HERE ###
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [40]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        ### START CODE HERE ### (≈ 1 line of code)
        dZ = relu_backward(dA, activation_cache)
        ### END CODE HERE ###
        
    elif activation == "sigmoid":
        ### START CODE HERE ### (≈ 1 line of code)
        dZ = sigmoid_backward(dA, activation_cache)
        ### END CODE HERE ###
    
    ### START CODE HERE ### (≈ 1 line of code)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    ### END CODE HERE ###
    
    return dA_prev, dW, db